**Mount Google Drive first!**

In [11]:
import cv2 as cv
import sys
import numpy as np
import os.path
import time

In [12]:
# Initialize the parameters
confThreshold = 0.6  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

inpWidth = 608 #416 #Width of network's input image
inpHeight =  608 #416 #Height of network's input image

# Load names of classes
classesFile = '/content/drive/MyDrive/darknet/lp.names'

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.

modelConfiguration = '/content/drive/MyDrive/darknet/license-plate.cfg'
modelWeights = '/content/drive/MyDrive/darknet/backup/license-plate_final.weights'

image_path = False
video_path = '/content/drive/MyDrive/testvideo.mp4'
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


In [13]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [32]:
# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom, framecount, path):
    # Draw a bounding box.
    #    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    if framecount % 10 == 0:
      timestamp = str(time.time())[6:10] + str(time.time())[11:14]
      cropped_frame = frame[top:bottom,left:right]
      try:
        cv.imwrite(os.path.join(path , '{:06d}'.format(framecount) + '_' + timestamp + '.jpg'), cropped_frame)
      except:
        print(f'Frame {framecount} skipped')
    cv.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)
    label = '%.2f' % conf

    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (0, 0, 255), cv.FILLED)
    cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)

In [33]:
# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs, framecount):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    detection_count = 0
    for out in outs:
        # print("out.shape : ", out.shape)
        for detection in out:
            #if detection[4]>0.001:
            scores = detection[5:]
            classId = np.argmax(scores)
            #if scores[classId]>confThreshold:
            confidence = scores[classId]
            if detection[4]>confThreshold:
                # print(detection[4], " - ", scores[classId], " - th : ", confThreshold)
                # print(detection)
                detection_count += 1
                # print(f"Objects detected in frame {framecount} = {detection_count}")
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height, framecount, path = '/content/drive/MyDrive/imgs')

In [34]:
# Process inputs
# winName = 'Deep learning object detection in OpenCV'
# cv.namedWindow(winName, cv.WINDOW_NORMAL)

outputFile = "yolo_out_gpu.avi"
if (image_path):
    # Open the image file
    if not os.path.isfile(image_path):
        print("Input image file ", image_path, " doesn't exist")
        sys.exit(1)
    cap = cv.VideoCapture(image_path)
    outputFile = image_path[:-4]+'_yolo_out_gpu.jpg'
elif (video_path):
    # Open the video file
    if not os.path.isfile(video_path):
        print("Input video file ", video_path, " doesn't exist")
        sys.exit(1)
    cap = cv.VideoCapture(video_path)
    outputFile = video_path[:-4]+'_yolo_out_gpu.avi'
else:
    # Webcam input
    cap = cv.VideoCapture(0)

# Get the video writer initialized to save the output video
if (not image_path):
    vid_writer = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc('M','J','P','G'), 30, (round(cap.get(cv.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))

framecount = 0
inf_time = 0 
start = time.time()
while True:

    # get frame from the video
    hasFrame, frame = cap.read()

    # Stop the program if reached end of video
    if not hasFrame:
        print("Done processing !!!")
        print("Output file is stored as ", outputFile)
        # cv.waitKey(3000)
        break
    framecount += 1
    # Create a 4D blob from a frame.
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(getOutputsNames(net))

    # Remove the bounding boxes with low confidence
    postprocess(frame, outs, framecount)

    # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
    t, _ = net.getPerfProfile()
    # label = 'Inference time: %.4f s' % (t/ cv.getTickFrequency())
    inf_time += t  / cv.getTickFrequency()
    #cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
    # print(label)
    
    # Write the frame with the detection boxes
    if (image_path):
        cv.imwrite(outputFile, frame.astype(np.uint8))
    else:
        vid_writer.write(frame.astype(np.uint8))
end = time.time()

inference_time = [int(inf_time // 60) , int(inf_time % 60)]
total_time = [int((end - start) // 60) , int((end - start) % 60)] 
print('\n')
print(f"Time taken for program: {total_time[0]} minutes, {total_time[1]} seconds approx.")
print(f"Time taken for inference in CPU: {inference_time[0]} minutes, {inference_time[1]} seconds approx.")
print('\n')
print(f"Total frames processed = {framecount}")
print(f"Time taken per frame: {inf_time / framecount:.3f} seconds")

KeyboardInterrupt: ignored

In [ ]:
# !cp  "/content/testvideo_yolo_out_cpu.avi"   /content/drive/MyDrive/colab_outputs